In [66]:
!pip install torch
!pip install torchvision

In [67]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
data = pd.read_csv('train.csv')

In [68]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using mps device


In [69]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) 

data_dev = data[0:1000].T
Y_val = data_dev[0]
X_val = data_dev[1:]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train= data_train[1:n]

In [71]:
class Neural_Network(nn.Module):
    def __init__(self): # Initalized when the object is called
        super().__init__() # Initalizes attributes of the parent class aswell
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 15),
            nn.ReLU(),
            nn.Linear(15, 10),
        )

    def forward(self, x) -> torch.Tensor:
        # Dimension of x should match the first dimension in the first layer of Sequential
        logits = self.linear_relu_stack(x)
        return logits

In [86]:
model = Neural_Network().to(device)
print(model)

Neural_Network(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=10, bias=True)
  )
)


In [87]:
loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.01)

In [88]:
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

TypeError: expected np.ndarray (got Tensor)

In [91]:
torch.manual_seed(42)

X_train = X_train.to(device)
X_val = X_val.to(device)
Y_train = Y_train.to(device)
Y_val = Y_val.to(device)

X_train = X_train.t()
X_val = X_val.t()

In [98]:
model = model.to(device)

epochs = 2

for epoch in range(epochs):
    model.train()
    Y_pred = model(X_train)
    loss = loss_fn(Y_pred, Y_train[epoch])
    
    # Zero grad optimizer
    optimizer.zero_grad()
    
    # Loss backward
    loss.backward()
    
    # Step the optimizer
    optimizer.step()

    model.eval()
    with torch.inference_mode():
        test_pred = model(X_val)

        # Calculate the loss
        test_loss = loss_fn(test_pred, Y_val[epoch])


    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")
    

Epoch: 0 | Train loss: 0.6212561726570129 | Test loss: 3.3797435760498047
Epoch: 10 | Train loss: 5.372744083404541 | Test loss: 6.371743679046631
Epoch: 20 | Train loss: 5.364744186401367 | Test loss: 4.363743782043457
Epoch: 30 | Train loss: 6.358744144439697 | Test loss: 4.357743740081787
Epoch: 40 | Train loss: 1.3527441024780273 | Test loss: 6.351744174957275
Epoch: 50 | Train loss: 2.3447444438934326 | Test loss: 8.343743324279785
Epoch: 60 | Train loss: 0.33674490451812744 | Test loss: 6.335744380950928
Epoch: 70 | Train loss: 6.328745365142822 | Test loss: 3.327744483947754
Epoch: 80 | Train loss: 1.320744514465332 | Test loss: 4.31974458694458
Epoch: 90 | Train loss: 6.3107452392578125 | Test loss: 2.309744358062744
Epoch: 100 | Train loss: 0.6972551941871643 | Test loss: 2.3037445545196533
Epoch: 110 | Train loss: 0.7052552103996277 | Test loss: 2.2957446575164795
Epoch: 120 | Train loss: 2.286745071411133 | Test loss: 2.2857449054718018
Epoch: 130 | Train loss: 2.27674508094

In [99]:
# Assuming that each column is a training example
# (3, 28, 28) -> (3, 784)
X = torch.rand(1,784, device=device)
logits = model(X)
print(logits.shape)
predicted_probability = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

torch.Size([1, 10])
Predicted class: tensor([8], device='mps:0')
